# Preprocessing
written by Tianpei Xie, Mar 14, 2016

In this script, we analyze the accident data, convert the categorical columns into indicators, transform the features. Then we combinine the accident data with person and vehicle data 

In [ ]:
%reset 
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
ifWrite = True


## Accident data
from ./train  and ./test.

In [ ]:
# read the accident file
print("Load accident data set")
acc_trn_df = pd.read_csv("./train/accident_train.csv")
#acc_trn_df.fillna(0, inplace=True)
#per_trn_df.fillna(0, inplace=True)
#veh_trn_df.fillna(0, inplace=True)

In [ ]:
np.shape(acc_trn_df)

In [ ]:
acc_trn_df.columns

## Data filtering and feature hashing
fillna, transfer categorical to numerical data, and count most frequent component

### Drop irrelevant features 

In [ ]:
#load the column names 
print("Drop irrelevant features")
acc_trn_org_columns = acc_trn_df.columns
acc_drop_list = ['YEAR', 'DAY', 'MONTH','CITY','TWAY_ID', 'RAIL','NOT_HOUR',\
                 'NOT_MIN','ARR_HOUR','ARR_MIN','HOSP_HR','HOSP_MN']
acc_trn_df.drop(acc_drop_list, axis = 1, inplace=True)
acc_trn_df.columns

In [ ]:
np.shape(acc_trn_df)

### Feature quantization for DAY_WEEK, HOUR and MINUTE, MILEPT
cut DAY_WEEK to be 0 (Weekdays) or 1 (Weekends)

In [ ]:
print("Features Quantizations")
acc_trn_df['COUNTY'] = acc_trn_df['COUNTY'].apply(lambda x: x if( x < 990) else np.nan)
acc_trn_df['DAY_WEEK'] = acc_trn_df['DAY_WEEK'].apply(lambda x: 1 if( x == 7 or x == 1 ) else 0)

acc_trn_latlon_df = acc_trn_df.loc[:,['LATITUDE','LONGITUD']]
acc_trn_latlon_df['LATITUDE'] = acc_trn_latlon_df['LATITUDE'].map(lambda x: x if(x <= 90 and x>= -90) else np.nan)
acc_trn_latlon_df['LONGITUD'] = acc_trn_latlon_df['LONGITUD'].map(lambda x: x if(x <= 180 and x>= -180) else np.nan)
acc_trn_df.loc[:,['LATITUDE','LONGITUD']] = acc_trn_latlon_df 

In [ ]:
temp_index = acc_trn_df.isnull().any(axis=1)
acc_nan_index = temp_index.index[temp_index==True]
len(acc_nan_index) 

Drop NAN rows

In [ ]:
acc_trn_dropout_df = acc_trn_df.loc[acc_nan_index,:]
if ifWrite:
    acc_trn_dropout_df.to_csv("./train/accident_train_dropout.csv")
acc_trn_dropout_ID = acc_trn_dropout_df['ID']

In [ ]:
acc_trn_df.dropna(axis =0, inplace=True)
np.shape(acc_trn_df)

quantization via quantile 

In [ ]:
out_hr, bins_hr = pd.qcut(x=acc_trn_df['HOUR'], q=[0, 0.1, 0.25, 0.5, 0.75, 0.9, 1.], labels=False, retbins=True)
acc_trn_df['HOUR'] = out_hr

In [ ]:
out_min, bins_min = pd.qcut(acc_trn_df['MINUTE'], q=[0, 0.1, 0.25, 0.5, 0.75, 0.9, 1.], labels=False, retbins=True)
acc_trn_df['MINUTE'] = out_min

In [ ]:
out_mile, bins_mile = pd.qcut(acc_trn_df['MILEPT'], q=[0, 0.1, 0.35, 0.5, 0.65, 0.8, 1.], labels=False, retbins=True)
acc_trn_df['MILEPT'] = out_mile

In [ ]:
acc_trn_df.groupby(['HOUR'])['ID'].count()

In [ ]:
acc_trn_df.groupby(['MILEPT'])['ID'].count()

### Vector quantization for (STATE, COUNTY), drop CITY

In [ ]:
temp_df = acc_trn_df.loc[:,['STATE','COUNTY']]

In [ ]:
sns.jointplot(x='STATE', y='COUNTY',data=temp_df) #scatterplot for (STATE, COUNTY) items, see clusters 

Vector quantization for [STATE, COUNTY]

In [ ]:
from sklearn.cluster import KMeans, DBSCAN
np.random.seed(123157) 
k_means = KMeans(n_clusters = 9, n_init= 8)
k_means.fit(temp_df.values)
state_county_labels = k_means.labels_
values = k_means.cluster_centers_.squeeze()

In [ ]:
# see histogram of a subset of 5000 samples 
randIndex = np.random.choice(len(state_county_labels), 5000)
temp = state_county_labels[randIndex]
sns.distplot(temp, kde=False, rug=True)  # see distribution of clusters

###  Extract (Latitude, Longitude) and make clustering

CLustering of (Latitude, Longitude)

In [ ]:
acc_trn_df.shape

In [ ]:
# see histogram of a subset of 5000 samples 
randIndex2 = np.random.choice(acc_trn_latlon_df.dropna(axis=0).index.tolist(), 50000)
temp2 = acc_trn_latlon_df.loc[randIndex2,:]

Use basemap to plot on the earth

from mpl_toolkits.basemap import Basemap
fig = plt.figure()
fig.set_size_inches(30,30)
m = Basemap(llcrnrlon=-130, llcrnrlat=np.min(temp2['LATITUDE']), 
            urcrnrlon=np.max(temp2['LONGITUD']), urcrnrlat=50, 
            #llcrnrlon=np.min(temp2['LONGITUD'])-20, llcrnrlat=np.min(temp2['LATITUDE'])-10, \
            #urcrnrlon=np.max(temp2['LONGITUD'])+20, urcrnrlat=np.max(temp2['LATITUDE'])+10,\
            projection='lcc', resolution='i', area_thresh=1, lat_1=-40, lat_2=42,lon_0=-85)
m.drawcoastlines()
m.drawstates()
m.drawcountries()
lons = temp2['LONGITUD'].values
lats = temp2['LATITUDE'].values
x, y = m(lons, lats)

m.scatter(x, y, s=20, marker='o', color='r')
plt.show()

Use DBSCAN - Density-Based Spatial Clustering of Applications with Noise to cluser

In [ ]:
n_acc= len(acc_trn_df.index)
rho = 0.2
np.ceil(n_acc*rho).astype(np.int64)

In [ ]:
dbscan = DBSCAN(eps=1, min_samples = 280, algorithm='kd_tree')
rho = 0.2
np.random.seed(12130) 
randIndex2 = np.random.choice(acc_trn_latlon_df.dropna(axis=0).index.tolist(), np.ceil(n_acc*rho).astype(np.int64))
subsample_trn = acc_trn_latlon_df.loc[randIndex2,:]
dbscan.fit(X=subsample_trn.values)
labels = dbscan.labels_
dbscan_values = dbscan.components_

In [ ]:
sns.distplot(labels[np.random.choice(len(labels), 1000)], kde=False, rug=True) 

In [ ]:
unique_labels = np.unique(labels)

In [ ]:
import matplotlib.colors as colors
import matplotlib.cm as cmx
jet = cm = plt.get_cmap('jet') 
cNorm  = colors.Normalize(vmin=-1, vmax=unique_labels[-1])
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

In [ ]:
from mpl_toolkits.basemap import Basemap
fig = plt.figure()
fig.set_size_inches(30,30)
m = Basemap(llcrnrlon=-130, llcrnrlat=np.min(temp2['LATITUDE']), 
            urcrnrlon=np.max(temp2['LONGITUD']), urcrnrlat=50, 
            #llcrnrlon=np.min(temp2['LONGITUD'])-20, llcrnrlat=np.min(temp2['LATITUDE'])-10, \
            #urcrnrlon=np.max(temp2['LONGITUD'])+20, urcrnrlat=np.max(temp2['LATITUDE'])+10,\
            projection='lcc', resolution='i', area_thresh=1, lat_1=-40, lat_2=42,lon_0=-85)
m.drawcoastlines()
m.drawstates()
m.drawcountries()
for c in np.unique(labels):
    lons = acc_trn_latlon_df.loc[randIndex2[labels==c],'LONGITUD'].values
    lats = acc_trn_latlon_df.loc[randIndex2[labels==c],'LATITUDE'].values
    x, y = m(lons, lats)
    colorVal = scalarMap.to_rgba(c)
    m.scatter(x, y, s=20, marker='o', color=colorVal, label=str(c))

#ax = plt.gca()
#handles, labels = ax.get_legend_handles_labels()  
plt.legend()
plt.show()

Predict cluster labels 

In [ ]:
n_batch = 10000
label_temp = np.zeros((n_acc,),dtype=np.int64)
np.ceil(n_acc/n_batch)

predict labels for all training samples

In [ ]:
for i in np.arange(np.ceil(n_acc/n_batch).astype(np.int64), dtype=np.int64):
    if i != np.ceil(n_acc/n_batch)-1:
        index_set = acc_trn_df.index[np.arange(start=i*n_batch, stop=(i+1)*n_batch)]
        temp_val = acc_trn_df.loc[index_set,['LATITUDE','LONGITUD']].values
        label_temp[i*n_batch:(i+1)*n_batch] = dbscan.fit_predict(temp_val)
    else:
        index_set = acc_trn_df.index[np.arange(start=i*n_batch, stop=n_acc)]
        temp_val = acc_trn_df.loc[index_set,['LATITUDE','LONGITUD']].values
        label_temp[i*n_batch:n_acc] = dbscan.fit_predict(temp_val)
        

In [ ]:
array_temp = np.column_stack((acc_trn_df['ID'].astype(np.int64).tolist(),label_temp))   
acc_trn_lat_lon_label_df = pd.DataFrame(data=array_temp, columns=["ID","LAT_LON_CLUSTER"])

In [ ]:
# save a series
array_temp = np.column_stack((acc_trn_df['ID'].astype(np.int64).tolist(),state_county_labels))   
acc_trn_state_county_df= pd.DataFrame(data=array_temp, columns=["ID", "STATE_COUNTY"])

In [ ]:
acc_trn_lat_lon_label_df['LAT_LON_CLUSTER'].value_counts()

Save for clusters 

In [ ]:
if ifWrite:
    acc_trn_lat_lon_label_df.to_csv('./lat_lon_label.csv')
    acc_trn_state_county_df.to_csv('./state_county_label.csv')

### Convert HOUR, MINUTE, WEEK_DAYS, (STATE, COUNTY) into indicators,  merge dataframe
merge with (STATE, COUNTY) cluster labels

In [ ]:
acc_trn_df_ext = acc_trn_df.merge(right=acc_trn_state_county_df, how='left', on='ID')

merge with (LATITUDE, LONGITUD) cluster labels

In [ ]:
acc_trn_df_ext = acc_trn_df_ext.merge(right=acc_trn_lat_lon_label_df, how='left', on='ID')

drop (STATE, COUNTY, LATITUDE, LONGITUD)

In [ ]:
acc_trn_df_ext.drop(['STATE' , 'COUNTY'], axis=1, inplace=True)

In [ ]:
np.shape(acc_trn_df_ext)

get dummy variables for the following list of features

In [ ]:
print("get dummy variables for the following list of features")
cat_list = ['DAY_WEEK','HOUR', 'MINUTE', 'MILEPT','ROAD_FNC','ROUTE','SP_JUR',\
             'MAN_COLL','REL_ROAD','LGT_COND','WEATHER', \
             'SCH_BUS']
add_dummies = []
column_sel = 'HOUR'
for i,column_sel in enumerate(cat_list): 
    add_dummies = []
    temp_dummy  = pd.get_dummies(acc_trn_df_ext[column_sel]).astype(np.int64)
    column_name = ['ID']
    add_dummies  = np.column_stack((acc_trn_df['ID'].astype(np.int64).tolist(), \
                              temp_dummy.values))
    
    for cc in temp_dummy.columns:
        column_name.append(column_sel+'_'+str(cc))

    acc_trn_df_ext = acc_trn_df_ext.merge(right=pd.DataFrame(data=add_dummies, columns=column_name), \
                                      how='left', on='ID')    

In [ ]:
acc_trn_df_ext.drop(cat_list, axis=1, inplace=True)

In [ ]:
np.shape(acc_trn_df_ext)

In [ ]:
#acc_trn_df_ext.columns.tolist()

## Transform the test samples
Following the same step as above

In [ ]:
print("Load test data: ")
#Load data 
acc_tst_df  = pd.read_csv("./test/accident_test.csv")

#fillna
acc_tst_df.fillna(0, inplace=True)

print("Feature filtering and quanitization...")

In [ ]:

acc_drop_list = ['DAY', 'MONTH','CITY','TWAY_ID', 'RAIL','NOT_HOUR',\
                 'NOT_MIN','ARR_HOUR','ARR_MIN','HOSP_HR','HOSP_MN']
acc_tst_df.drop(acc_drop_list, axis = 1, inplace=True)
acc_tst_df.columns

For training set

Index([u'ID', u'STATE', u'VE_FORMS', u'PEDS', u'PERSONS', u'COUNTY',
       u'DAY_WEEK', u'HOUR', u'MINUTE', u'NHS', u'ROAD_FNC', u'ROUTE',
       u'MILEPT', u'LATITUDE', u'LONGITUD', u'SP_JUR', u'HARM_EV', u'MAN_COLL',
       u'REL_ROAD', u'LGT_COND', u'WEATHER', u'SCH_BUS', u'CF1', u'CF2',
       u'CF3', u'FATALS', u'DRUNK_DR'],
      dtype='object')

training set (287586, 27)

In [ ]:
np.shape(acc_tst_df)

In [ ]:
#acc_tst_df['COUNTY'] = acc_tst_df['COUNTY'].apply(lambda x: x if( x < 990) else np.nan)
acc_tst_df['DAY_WEEK'] = acc_tst_df['DAY_WEEK'].apply(lambda x: 1 if( x == 7 or x == 1 ) else 0)

acc_tst_latlon_df = acc_tst_df.loc[:,['LATITUDE','LONGITUD']]
#acc_tst_latlon_df['LATITUDE'] = acc_tst_latlon_df['LATITUDE'].map(lambda x: x if(x <= 90 and x>= -90) else np.nan)
#acc_tst_latlon_df['LONGITUD'] = acc_tst_latlon_df['LONGITUD'].map(lambda x: x if(x <= 180 and x>= -180) else np.nan)
#acc_tst_df.loc[:,['LATITUDE','LONGITUD']] = acc_tst_latlon_df 

In [ ]:
np.shape(acc_tst_df)

In [ ]:
out_hr_tst = pd.cut(x=acc_tst_df['HOUR'], bins=bins_hr, labels=False)
acc_tst_df['HOUR'] = out_hr_tst.fillna(value=0,axis=0).astype(np.int64)

out_min_tst = pd.cut(x=acc_tst_df['MINUTE'], bins=bins_min, labels=False)
acc_tst_df['MINUTE'] = out_min_tst.fillna(value=0,axis=0).astype(np.int64)

out_mile_tst = pd.cut(x=acc_tst_df['MILEPT'], bins=bins_mile, labels=False)
acc_tst_df['MILEPT'] = out_mile_tst.fillna(value=0,axis=0).astype(np.int64)

In [ ]:
tst_state_county_labels = k_means.fit_predict(acc_tst_df.loc[:,['STATE','COUNTY']].values)
tst_label = dbscan.fit_predict(acc_tst_df.loc[:,['LATITUDE','LONGITUD']].values)
unique, counts = np.unique(tst_label, return_counts=True)
print(np.asarray((unique, counts)).T)

In [ ]:
array_temp_tst = np.column_stack((acc_tst_df['ID'].astype(np.int64).tolist(), tst_label))
acc_tst_lat_lon_label_df = pd.DataFrame(data=array_temp_tst, columns=["ID","LAT_LON_CLUSTER"])

In [ ]:
array_temp_tst = np.column_stack((acc_tst_df['ID'].astype(np.int64).tolist(), tst_state_county_labels))
acc_tst_state_county_df= pd.DataFrame(data=array_temp_tst, columns=["ID", "STATE_COUNTY"])
%xdel array_temp_tst

In [ ]:
acc_tst_df_ext = acc_tst_df.merge(right=acc_tst_state_county_df, how='left', on='ID')
acc_tst_df_ext = acc_tst_df_ext.merge(right=acc_tst_lat_lon_label_df, how='left', on='ID')
acc_tst_df_ext.drop(['STATE' , 'COUNTY'], axis=1, inplace=True)
np.shape(acc_tst_df_ext)

training (277996, 25)

In [ ]:
cat_list = ['DAY_WEEK','HOUR', 'MINUTE', 'MILEPT','ROAD_FNC','ROUTE','SP_JUR',\
             'MAN_COLL','REL_ROAD','LGT_COND','WEATHER', \
             'SCH_BUS']
add_dummies = []
column_sel = 'HOUR'
for i,column_sel in enumerate(cat_list): 
    add_dummies = []
    temp_dummy  = pd.get_dummies(acc_tst_df_ext[column_sel]).astype(np.int64)
    column_name = ['ID']
    add_dummies  = np.column_stack((acc_tst_df['ID'].astype(np.int64).tolist(), \
                              temp_dummy.values))
    for cc in temp_dummy.columns:
        column_name.append(column_sel+'_'+str(cc))

    acc_tst_df_ext = acc_tst_df_ext.merge(right=pd.DataFrame(data=add_dummies, columns=column_name), \
                                      how='left', on='ID')    

In [ ]:
acc_tst_df_ext.drop(cat_list, axis=1, inplace=True)

In [ ]:
np.shape(acc_tst_df_ext)

training set (277996, 115)

In [ ]:
acc_tst_df_ext.columns[~acc_tst_df_ext.columns.isin(acc_trn_df_ext.columns)]

In [ ]:
acc_trn_df_ext.columns[~acc_trn_df_ext.columns.isin(acc_tst_df_ext.columns)]

combine and expand the training and test features to make them consistent

In [ ]:
n_acc_trn = acc_trn_df_ext.shape[0]
n_acc_tst = acc_tst_df_ext.shape[0]
temp4 = acc_trn_df_ext.merge(right=acc_tst_df_ext, how='outer')

extract extended training and test data set

In [ ]:
acc_trn_df_ext = temp4.iloc[np.arange(start=0,stop=n_acc_trn),:].fillna(0,axis=1)

acc_tst_df_ext = temp4.iloc[np.arange(start=n_acc_trn,stop=n_acc_trn+n_acc_tst),:].drop('DRUNK_DR',axis=1).fillna(0,axis=1)

In [ ]:
%xdel temp4
n_acc_trn = acc_trn_df_ext.shape[0]
acc_trn_df_ext.shape

In [ ]:
acc_tst_df_ext.shape

save the accident data

In [ ]:
Ydata = acc_trn_df_ext['DRUNK_DR'].values
Ydata_df = acc_trn_df_ext.loc[:, ['ID','DRUNK_DR']]

acc_trn_df_ext.drop('DRUNK_DR',axis=1, inplace=True)

In [ ]:
if ifWrite:
    print("save to files for both training and testing data ...")
    try:
        os.remove('./train/accident_train_ext.csv')
        os.remove('./test/accident_test_ext.csv')
    except OSError:
        pass
    acc_trn_df_ext.to_csv('./train/accident_train_ext.csv')
    Ydata_df.to_csv('./train/labels_ext.csv')
    acc_tst_df_ext.to_csv('./test/accident_test_ext.csv')

In [ ]:
if ifWrite:
    print("save to sql database for training set  ...")
    conn = sqlite3.connect("./train/joint_accident_person_vehicle.db")
    n_batch = 1000

    for i in np.arange(np.ceil(n_acc_trn/n_batch).astype(np.int64), dtype=np.int64):
        if i == 0:
            temp = pd.read_csv('./train/accident_train_ext.csv', nrows=n_batch, index_col=0)
            temp.to_sql("accident_trans", conn, if_exists= "replace", index=True)
        else:
            if i!= np.ceil(n_per_veh/n_batch).astype(np.int64)-1:
                temp = pd.read_csv('./train/accident_train_ext.csv', \
                               skiprows= range(1, i*n_batch+1),\
                               nrows=n_batch, index_col=0)
                temp.to_sql("accident_trans", conn, if_exists= "append", index=True)
            else:
                temp = pd.read_csv('./train/accident_train_ext.csv', \
                               skiprows= range(1, i*n_batch+1),\
                               nrows=n_batch, index_col=0)
                temp.to_sql("accident_trans", conn, if_exists= "append", index=True)
    conn.close()

In [ ]:
%xdel  acc_trn_df
%xdel  acc_tst_df
acc_trn_df_ext.shape

In [ ]:
%xdel acc_trn_lat_lon_label_df
%xdel acc_trn_state_county_df
%xdel acc_tst_latlon_df
%xdel acc_tst_state_county_df
%xdel acc_tst_lat_lon_label_df
%xdel label_temp
%xdel tst_state_county_labels
%xdel state_county_labels

##  Person dataset 

We merge from person dataset. Note that more than one person is involved in the accident, they will span multiple rows.

We now work on person data itself

First, we delete duplicates columns in person dataset

In [ ]:
# read the person file
per_trn_df = pd.read_csv("./train/person_train.csv")
# read the test file
per_tst_df = pd.read_csv("./test/person_test.csv")
per_tst_df.fillna(0, inplace=True)
per_trn_org_columns = per_trn_df.columns

In [ ]:
duplicate_list = []
per_trn_org_columns[per_trn_org_columns.isin(acc_trn_org_columns)].drop('ID')

In [ ]:
duplicate_list = per_trn_org_columns[per_trn_org_columns.isin(acc_trn_org_columns)].drop('ID')
per_trn_df_ext = per_trn_df.drop(duplicate_list, axis=1)
per_tst_df_ext = per_tst_df.drop(duplicate_list, axis=1)

In [ ]:
irrelevant_cols = ['CERT_NO','EMER_USE','AIR_BAG','DEATH_MO','DEATH_DA','DEATH_HR','DEATH_MN','DEATH_TM',\
                   'EJ_PATH', 'EXTRICAT','ROLLOVER','LAG_HRS','LAG_MINS','FIRE_EXP']
per_trn_df_ext.drop(irrelevant_cols, axis=1, inplace=True)
per_tst_df_ext.drop(irrelevant_cols, axis=1, inplace=True)

In [ ]:
per_trn_df_ext.shape

Delete the rows that correspond to the deleted rows in accident data 

In [ ]:
per_nan_index = per_trn_df_ext.index[per_trn_df_ext['ID'].isin(acc_trn_dropout_ID)]
per_trn_dropout_df = per_trn_df_ext.loc[per_nan_index,:]
if ifWrite:
    per_trn_dropout_df.to_csv("./train/person_train_dropout.csv")
per_trn_df_ext.drop(per_nan_index, axis=0)
per_trn_df_ext.shape

Quantization for AGE,  PER_TYP

Note (PER_NO, VEH_NO) associates each person with a vehicle. For example, ID=77, has 4 people involved 
given as 
\begin{table}[ht]
  \begin{center}
    \begin{tabular}{cc}
      \toprule
        PER_NO & VEH_NO \\
           $1$   &   $1$\\    
           $1$   &   $2$\\    
           $2$   &   $2$\\    
           $3$   &   $2$\\    
      \bottomrule
    \end{tabular}
  \end{center}
\end{table}

In [ ]:
out_age, bins_age = pd.qcut(x=per_trn_df_ext['AGE'], q=[0, 0.1, 0.25, 0.5, 0.75, 0.9, 1.], labels=False, retbins=True)
per_trn_df_ext['AGE'] = out_age

In [ ]:
#out_per, bins_per = pd.qcut(x=per_trn_df_ext['PER_NO'], q=[0, 0.75, 0.9, 1.], labels=False, retbins=True)
#per_trn_df_ext['PER_NO'] = out_per

In [ ]:
#out_veh, bins_veh = pd.qcut(x=per_trn_df_ext['VEH_NO'], q=[0, 0.5, 0.9, 1.], labels=False, retbins=True)
#per_trn_df_ext['VEH_NO'] = out_veh

In [ ]:
per_trn_df_ext['PER_TYP'] = per_trn_df_ext['PER_TYP'].apply(lambda x: x if (x not in [-1, 5,6,7,8,19]) else 5 )
per_tst_df_ext['PER_TYP'] = per_tst_df_ext['PER_TYP'].apply(lambda x: x if (x not in [-1, 5,6,7,8,19]) else 5 )

In [ ]:
out_age_tst = pd.cut(x=per_tst_df_ext['AGE'], bins=bins_age, labels=False)
per_tst_df_ext['AGE'] = out_age_tst.fillna(value=0,axis=0).astype(np.int64)

Get dummy variables for categorical features

In [ ]:
per_trn_df_ext.shape

In [ ]:
temp_index_per = per_trn_df_ext.index.tolist()
per_trn_df_ext['EXTRA_INDEX'] = per_trn_df_ext.index.tolist() #the ID is duplicate
cat_list_per = ['AGE','SEX', 'PER_TYP', 'SPEC_USE' ,'SEAT_POS','DOA','HISPANIC','RACE', 'IMPACT1',\
                'WORK_INJ', 'INJ_SEV']
add_dummies = []
for column_sel in cat_list_per: 
    add_dummies = []
    temp_dummy  = pd.get_dummies(per_trn_df_ext[column_sel]).astype(np.int64)
    column_name = ['EXTRA_INDEX']
    add_dummies  = np.column_stack((per_trn_df_ext.index.tolist(), \
                              temp_dummy.values))
    for cc in temp_dummy.columns:
        column_name.append(column_sel+'_'+str(cc))

    per_trn_df_ext = per_trn_df_ext.merge(right=pd.DataFrame(data=add_dummies, columns=column_name), \
                                      how='left', on='EXTRA_INDEX')    

In [ ]:
per_trn_df_ext.shape

Delete redundant features

In [ ]:
#per_trn_df_ext['EXTRA_INDEX']
#%xdel  acc_trn_df,  acc_tst_df
n_per_trn = per_trn_df_ext.shape[0]
if ifWrite:
    try:
        os.remove('./train/person_temp.csv')
    except OSError:
        pass
    per_trn_df_ext.to_csv('./train/person_temp.csv')
%xdel per_trn_df_ext

In [ ]:
n_batch = 10000
for i in np.arange(np.ceil(n_per_trn/n_batch).astype(np.int64), dtype=np.int64):
    if i == 0:
        temp = pd.read_csv('./train/person_temp.csv', nrows=n_batch, index_col=0)
        temp.drop('EXTRA_INDEX',axis=1,inplace=True)
        temp.drop(cat_list_per,axis=1,inplace=True)
        per_trn_df_ext = temp
    else:
        temp = pd.read_csv('./train/person_temp.csv', \
                               skiprows= range(1, i*n_batch+1),\
                               nrows=n_batch, index_col=0)
        temp.drop('EXTRA_INDEX',axis=1,inplace=True)
        temp.drop(cat_list_per,axis=1,inplace=True)
        per_trn_df_ext = pd.concat([per_trn_df_ext, temp])
#per_trn_df_ext.index = temp_index_per

In [ ]:
per_trn_df_ext.shape

Tranform the test data

In [ ]:
temp_index_per_tst = per_tst_df_ext.index.tolist()
per_tst_df_ext['EXTRA_INDEX'] = per_tst_df_ext.index.tolist() #the ID is duplicate
cat_list_per = ['AGE','SEX', 'PER_TYP', 'SPEC_USE' ,'SEAT_POS','DOA','HISPANIC','RACE', 'IMPACT1',\
                'WORK_INJ', 'INJ_SEV']
add_dummies = []
for column_sel in cat_list_per: 
    add_dummies = []
    temp_dummy  = pd.get_dummies(per_tst_df_ext[column_sel]).astype(np.int64)
    column_name = ['EXTRA_INDEX']
    add_dummies  = np.column_stack((per_tst_df_ext.index.tolist(), \
                              temp_dummy.values))
    for cc in temp_dummy.columns:
        column_name.append(column_sel+'_'+str(cc))

    per_tst_df_ext = per_tst_df_ext.merge(right=pd.DataFrame(data=add_dummies, columns=column_name), \
                                      how='left', on='EXTRA_INDEX')  
per_tst_df_ext.shape    

In [ ]:
n_per_tst = per_tst_df_ext.shape[0]
per_tst_df_ext.drop('EXTRA_INDEX',axis=1,inplace=True)
per_tst_df_ext.drop(cat_list_per,axis=1,inplace=True)

Make sure consistency

In [ ]:
n_trn_trail = 2; n_tst_trial = 2
temp_tst = per_tst_df_ext.loc[range(0,n_tst_trial),:]
temp_trn = per_trn_df_ext.loc[range(n_per_trn-n_trn_trail,n_per_trn),:]

temp = temp_trn.merge(right=temp_tst, how='outer')
temp4 =  per_trn_df_ext.merge(right=temp, how='outer')
#temp4 = per_trn_df_ext.merge(right=per_tst_df_ext, how='outer')
per_trn_df_ext = temp4.iloc[np.arange(start=0,stop=n_per_trn),:].fillna(0,axis=1)
%xdel temp4

temp4 =  temp.merge(right=per_tst_df_ext, how='outer')
per_tst_df_ext = temp4.iloc[np.arange(start=n_trn_trail,stop=temp4.shape[0]),:].fillna(0,axis=1)
per_tst_df_ext.index = temp_index_per_tst

In [ ]:
%xdel temp4
per_trn_df_ext.shape
#%xdel per_trn_df_ext

(732461, 143)

In [ ]:
per_tst_df_ext.shape

In [ ]:
if ifWrite:
    try:
        os.remove('./train/person_train_ext.csv')
        os.remove('./test/person_test_ext.csv')
    except OSError:
        pass
    per_trn_df_ext.to_csv('./train/person_train_ext.csv')
    per_tst_df_ext.to_csv('./test/person_test_ext.csv')

In [ ]:
%xdel per_trn_df_ext
%xdel per_tst_df_ext
%xdel temp_index_per_tst
%xdel temp_index_per
%xdel temp
%xdel add_nummies
%xdel temp_dummy

In [ ]:
if ifWrite:
    conn = sqlite3.connect("./train/joint_accident_person_vehicle.db")
    n_batch = 1000

    for i in np.arange(np.ceil(n_per_trn/n_batch).astype(np.int64), dtype=np.int64):
        if i == 0:
            temp = pd.read_csv('./train/person_train_ext.csv', nrows=n_batch, index_col=0)
            temp.to_sql("person_trans", conn, if_exists= "replace", index=True)
        else:
            if i!= np.ceil(n_per_veh/n_batch).astype(np.int64)-1:
                temp = pd.read_csv('./train/person_train_ext.csv', \
                               skiprows= range(1, i*n_batch+1),\
                               nrows=n_batch, index_col=0)
                temp.to_sql("person_trans", conn, if_exists= "append", index=True)
            else:
                temp = pd.read_csv('./train/person_train_ext.csv', \
                               skiprows= range(1, i*n_batch+1),\
                               nrows=n_batch, index_col=0)
                temp.to_sql("person_trans", conn, if_exists= "append", index=True)
    conn.close()

## Vehicle data
Process the individul vehicle data

First, remove duplicates

In [ ]:
# read the vehicle file
veh_trn_df = pd.read_csv("./train/vehicle_train.csv")
veh_tst_df = pd.read_csv("./test/vehicle_test.csv")
veh_tst_df.fillna(0, inplace=True)
veh_trn_org_columns = veh_trn_df.columns

In [ ]:
veh_trn_org_columns[veh_trn_org_columns.isin(acc_trn_org_columns)].drop('ID')

In [ ]:
duplicate_list = veh_trn_org_columns[veh_trn_org_columns.isin(acc_trn_org_columns)].drop('ID')
veh_trn_df_ext = veh_trn_df.drop(duplicate_list, axis=1)
veh_tst_df_ext = veh_tst_df.drop(duplicate_list, axis=1)

Delete rows that corresponds to deleted ID in accident data

In [ ]:
veh_nan_index = veh_trn_df_ext.index[veh_trn_df_ext['ID'].isin(acc_trn_dropout_ID)]
veh_trn_dropout_df = veh_trn_df_ext.loc[veh_nan_index,:]
if ifWrite:
    veh_trn_dropout_df.to_csv("./train/vehicle_train_dropout.csv")
veh_trn_df_ext.drop(veh_nan_index, axis=0)
veh_trn_df_ext.shape

In [ ]:
irrelevant_cols = ['BUS_USE','DR_ZIP','VIN','VIN_1','VIN_2','VIN_3','VIN_4',\
                   'VIN_5','VIN_6','VIN_7','VIN_8','VIN_9',\
                   'VIN_10','VIN_11','VIN_12', 'TOW_VEH','ROLLOVER', 'DEFORMED', \
                   'MCARR_ID','EMER_USE','CARGO_BT','DEATHS','FIRE_EXP','UNDERIDE','LAST_YR','FIRST_YR']
veh_trn_df_ext.drop(irrelevant_cols, axis=1, inplace=True)
veh_tst_df_ext.drop(irrelevant_cols, axis=1, inplace=True)

In [ ]:
out_speed, bins_speed = pd.qcut(x=veh_trn_df_ext['TRAV_SP'], q=[0, 0.1, 0.25, 0.5, 0.75, 0.9, 1.], labels=False, retbins=True)
veh_trn_df_ext['TRAV_SP'] = out_speed

In [ ]:
out_speed_tst = pd.cut(x=veh_tst_df_ext['TRAV_SP'], bins=bins_speed, labels=False)
veh_tst_df_ext['TRAV_SP'] = out_speed_tst.fillna(value=0,axis=0).astype(np.int64)

In [ ]:
out_pre_acc, bins_pre_acc = pd.qcut(x=veh_trn_df_ext['PREV_ACC'], q=[0, 0.8, 0.9, 1.], labels=False, retbins=True)
veh_trn_df_ext['PREV_ACC'] = out_pre_acc

In [ ]:
out_pre_acc_tst = pd.cut(x=veh_tst_df_ext['PREV_ACC'], bins=bins_pre_acc, labels=False)
veh_tst_df_ext['PREV_ACC'] = out_pre_acc_tst.fillna(value=0,axis=0).astype(np.int64)

In [ ]:
out_pre_sus, bins_pre_sus = pd.qcut(x=veh_trn_df_ext['PREV_SUS'], q=[0, 0.9, 0.95, 1.], labels=False, retbins=True)
veh_trn_df_ext['PREV_SUS'] = out_pre_sus

In [ ]:
out_pre_sus_tst = pd.cut(x=veh_tst_df_ext['PREV_SUS'], bins=bins_pre_sus, labels=False)
veh_tst_df_ext['PREV_SUS'] = out_pre_sus_tst.fillna(value=0,axis=0).astype(np.int64)

In [ ]:
out_pre_dwi, bins_pre_dwi = pd.qcut(x=veh_trn_df_ext['PREV_DWI'], q=[0, 0.95, 1.], labels=False, retbins=True)
veh_trn_df_ext['PREV_DWI'] = out_pre_dwi

In [ ]:
out_pre_dwi_tst = pd.cut(x=veh_tst_df_ext['PREV_DWI'], bins=bins_pre_dwi, labels=False)
veh_tst_df_ext['PREV_DWI'] = out_pre_dwi_tst.fillna(value=0,axis=0).astype(np.int64)

In [ ]:
out_pre_speed, bins_pre_speed = pd.qcut(x=veh_trn_df_ext['PREV_SPD'], q=[0, 0.9, 0.95, 1.], labels=False, retbins=True)
veh_trn_df_ext['PREV_SPD'] = out_pre_speed

In [ ]:
out_pre_speed_tst = pd.cut(x=veh_tst_df_ext['PREV_SPD'], bins=bins_pre_speed, labels=False)
veh_tst_df_ext['PREV_SPD'] = out_pre_speed_tst.fillna(value=0,axis=0).astype(np.int64)

In [ ]:
out_pre_other, bins_pre_other = pd.qcut(x=veh_trn_df_ext['PREV_OTH'], q=[0, 0.9, 0.95, 1.], labels=False, retbins=True)
veh_trn_df_ext['PREV_OTH'] = out_pre_other

In [ ]:
out_pre_other_tst = pd.cut(x=veh_tst_df_ext['PREV_OTH'], bins=bins_pre_other, labels=False)
veh_tst_df_ext['PREV_OTH'] = out_pre_other_tst.fillna(value=0,axis=0).astype(np.int64)

In [ ]:
veh_trn_df_ext.shape

In [ ]:
temp_index_veh = veh_trn_df_ext.index.tolist()
veh_trn_df_ext['EXTRA_INDEX'] = veh_trn_df_ext.index.tolist() #the ID is duplicate
cat_list_per = ['OWNER', 'J_KNIFE', 'SPEC_USE', 'TRAV_SP', 'IMPACT1', 'M_HARM' ,'PREV_ACC', 'PREV_SUS' , \
                'PREV_DWI' , 'PREV_SPD' ,'PREV_OTH', 'L_STATUS', 'CDL_STAT', 'L_ENDORS', 'L_COMPL',\
                'L_RESTRI', 'LAST_MO']
add_dummies = []
for column_sel in cat_list_per: 
    add_dummies = []
    temp_dummy  = pd.get_dummies(veh_trn_df_ext[column_sel]).astype(np.int64)
    column_name = ['EXTRA_INDEX']
    add_dummies  = np.column_stack((veh_trn_df_ext.index.tolist(), \
                              temp_dummy.values))
    for cc in temp_dummy.columns:
        column_name.append(column_sel+'_'+str(cc))

    veh_trn_df_ext = veh_trn_df_ext.merge(right=pd.DataFrame(data=add_dummies, columns=column_name), \
                                      how='left', on='EXTRA_INDEX')  

In [ ]:
veh_trn_df_ext.shape

In [ ]:
if ifWrite:
    try:
        os.remove('./train/vehicle_temp.csv')
    except OSError:
        pass

    n_veh_trn = veh_trn_df_ext.shape[0]
    veh_trn_df_ext.to_csv('./train/vehicle_temp.csv')
%xdel veh_trn_df_ext

In [ ]:
n_batch = 10000
for i in np.arange(np.ceil(n_veh_trn/n_batch).astype(np.int64), dtype=np.int64):
    if i == 0:
        temp = pd.read_csv('./train/vehicle_temp.csv', nrows=n_batch, index_col=0)
        temp.drop('EXTRA_INDEX',axis=1,inplace=True)
        temp.drop(cat_list_per,axis=1,inplace=True)
        veh_trn_df_ext = temp
    else:
        temp = pd.read_csv('./train/vehicle_temp.csv', \
                               skiprows= range(1, i*n_batch+1),\
                               nrows=n_batch, index_col=0)
        temp.drop('EXTRA_INDEX',axis=1,inplace=True)
        temp.drop(cat_list_per,axis=1,inplace=True)
        veh_trn_df_ext = pd.concat([veh_trn_df_ext, temp])
#veh_trn_df_ext.index = temp_index_veh

In [ ]:
veh_trn_df_ext

In [ ]:
#veh_trn_df_ext.drop('EXTRA_INDEX',axis=1,inplace=True)
#veh_trn_df_ext.drop(cat_list_per,axis=1,inplace=True)
veh_trn_df_ext.shape

In [ ]:
temp_index_veh_tst =  veh_tst_df_ext.index.tolist() 
veh_tst_df_ext['EXTRA_INDEX'] = veh_tst_df_ext.index.tolist() #the ID is duplicate
cat_list_per = ['OWNER', 'J_KNIFE', 'SPEC_USE', 'TRAV_SP', 'IMPACT1', 'M_HARM' ,'PREV_ACC', 'PREV_SUS' , \
                'PREV_DWI' , 'PREV_SPD' ,'PREV_OTH', 'L_STATUS', 'CDL_STAT', 'L_ENDORS', 'L_COMPL',\
                'L_RESTRI', 'LAST_MO']
add_dummies = []
for column_sel in cat_list_per: 
    add_dummies = []
    temp_dummy  = pd.get_dummies(veh_tst_df_ext[column_sel]).astype(np.int64)
    column_name = ['EXTRA_INDEX']
    add_dummies  = np.column_stack((veh_tst_df_ext.index.tolist(), \
                              temp_dummy.values))
    for cc in temp_dummy.columns:
        column_name.append(column_sel+'_'+str(cc))

    veh_tst_df_ext = veh_tst_df_ext.merge(right=pd.DataFrame(data=add_dummies, columns=column_name), \
                                      how='left', on='EXTRA_INDEX')  
veh_tst_df_ext.shape    

In [ ]:
%xdel add_dummies
n_veh_tst = veh_tst_df_ext.shape[0]
veh_tst_df_ext.drop('EXTRA_INDEX',axis=1,inplace=True)
veh_tst_df_ext.drop(cat_list_per,axis=1,inplace=True)

In [ ]:
n_trn_trail = 2; n_tst_trial = 2
temp_tst = veh_tst_df_ext.loc[np.arange(start=0,stop=n_tst_trial),:]
temp_trn = veh_trn_df_ext.loc[np.arange(start=n_veh_trn-n_trn_trail,stop=n_veh_trn),:]

temp = temp_trn.merge(right=temp_tst, how='outer')
temp4 =  veh_trn_df_ext.merge(right=temp, how='outer')
#temp4 = per_trn_df_ext.merge(right=per_tst_df_ext, how='outer')
veh_trn_df_ext = temp4.iloc[np.arange(start=0,stop=n_veh_trn),:].fillna(0,axis=1)
%xdel temp4

temp4 =  temp.merge(right=veh_tst_df_ext, how='outer')
veh_tst_df_ext = temp4.iloc[np.arange(start=n_trn_trail,stop=temp4.shape[0]),:].fillna(0,axis=1)
veh_tst_df_ext.index = temp_index_veh_tst

In [ ]:
%xdel temp4
veh_trn_df_ext.shape

In [ ]:
veh_tst_df_ext.shape

In [ ]:
if ifWrite:
    try:
        os.remove('./train/vehicle_train_ext.csv')
        os.remove('./test/vehicle_test_ext.csv')
    except OSError:
        pass
    veh_trn_df_ext.to_csv('./train/vehicle_train_ext.csv')
    veh_tst_df_ext.to_csv('./test/vehicle_test_ext.csv')

In [ ]:
#veh_tst_df_ext.to_csv('./test/vehicle_test_ext.csv')
%xdel veh_trn_df_ext
%xdel veh_tst_df_ext
%xdel temp
%xdel temp_index_veh_tst
%xdel temp_index_veh

In [ ]:
if ifWrite:
    conn = sqlite3.connect("./train/joint_accident_person_vehicle.db")
    n_batch = 1000

    for i in np.arange(np.ceil(n_veh_trn/n_batch).astype(np.int64), dtype=np.int64):
        if i == 0:
            temp = pd.read_csv('./train/vehicle_train_ext.csv', nrows=n_batch, index_col=0)
            temp.to_sql("vehicle_trans", conn, if_exists= "replace", index=True)
        else:
            if i!= np.ceil(n_per_veh/n_batch).astype(np.int64)-1:
                temp = pd.read_csv('./train/vehicle_train_ext.csv', \
                               skiprows= range(1, i*n_batch+1),\
                               nrows=n_batch, index_col=0)
                temp.to_sql("vehicle_trans", conn, if_exists= "append", index=True)
            else:
                temp = pd.read_csv('./train/vehicle_train_ext.csv', \
                               skiprows= range(1, i*n_batch+1),\
                               index_col=0)
                temp.to_sql("vehicle_trans", conn, if_exists= "append", index=True)
    conn.close()

In [ ]:
#%xdel temp

## Combine Accident and Person data

Load the transformed data from disk. Note that the total size is very large. Need to reset the kernel and release all memory to load the data. Make sure release memory after processing. 

In [ ]:
acc_trn_df_ext = pd.read_csv('./train/accident_train_ext.csv', index_col=0)

In [ ]:
acc_trn_df_ext.shape

In [ ]:
#%reset
per_trn_df_ext = pd.read_csv('./train/person_train_ext.csv', index_col=0)

In [ ]:
per_trn_df_ext.shape

In [ ]:
acc_per_joined_trn_df = pd.merge(acc_trn_df_ext, per_trn_df_ext, on='ID', how='inner')

In [ ]:
acc_per_joined_trn_df.shape
#joined_trn_df.shape

The joint data set filled the memory, so fill nan in batch mode

In [ ]:
n_batch = 10000
n_acc_per = acc_per_joined_trn_df.shape[0]

for i in np.arange(np.ceil(n_acc_per/n_batch).astype(np.int64), dtype=np.int64):
    if i != np.ceil(n_acc_per/n_batch).astype(np.int64)-1:
        acc_per_joined_trn_df.loc[np.arange(start=i*n_batch, stop=(i+1)*n_batch),:].fillna(0, inplace=True)
    else:
        acc_per_joined_trn_df.loc[np.arange(start=i*n_batch, stop=n_acc_per),:].fillna(0, inplace=True)

In [ ]:
if ifWrite:
    acc_per_joined_trn_df.to_csv('./train/joint_accident_person_train_ext.csv')
#joined_trn_df.fillna(0, inplace=True)

Write to sqlite

In [ ]:
if ifWrite:
    conn = sqlite3.connect("./train/joint_accident_person_vehicle.db")
    n_batch = 10000

    for i in np.arange(np.ceil(n_acc_per/n_batch).astype(np.int64), dtype=np.int64):
        if i == 0:
            acc_per_joined_trn_df.loc[np.arange(start=i*n_batch, stop=(i+1)*n_batch),:]\
                             .to_sql("accident_person", conn, if_exists= "replace", index=True)
        else:
            if i!= np.ceil(n_acc_per/n_batch).astype(np.int64)-1:
                acc_per_joined_trn_df.loc[np.arange(start=i*n_batch, stop=(i+1)*n_batch),:]\
                            .to_sql("accident_person", conn, if_exists= "append", index=True)
            else:
                acc_per_joined_trn_df.loc[np.arange(start=i*n_batch, stop=n_acc_per),:].\
                             to_sql("accident_person", conn, if_exists= "append", index=True)
    conn.close()

In [ ]:
%xdel acc_trn_df_ext
%xdel per_trn_df_ext
%xdel acc_per_joined_trn_df

### test data

In [ ]:
acc_tst_df_ext = pd.read_csv('./test/accident_test_ext.csv', index_col=0)
acc_tst_df_ext.shape

In [ ]:
per_tst_df_ext = pd.read_csv('./test/person_test_ext.csv', index_col=0)
per_tst_df_ext.shape

In [ ]:
acc_per_joined_tst_df = pd.merge(acc_tst_df_ext, per_tst_df_ext, on='ID', how='inner')
acc_per_joined_tst_df.shape

In [ ]:
acc_per_joined_tst_df.fillna(0, inplace=True)

In [ ]:
if ifWrite:
    acc_per_joined_tst_df.to_csv('./test/joint_accident_person_test_ext.csv')

In [ ]:
%xdel acc_tst_df_ext
%xdel per_tst_df_ext
%xdel acc_per_joined_tst_df

## Combine the Person and Vehicle data

In [ ]:
per_trn_df_ext = pd.read_csv('./train/person_train_ext.csv', index_col=0)
per_trn_df_ext.shape

In [ ]:
veh_trn_df_ext = pd.read_csv('./train/vehicle_train_ext.csv', index_col=0)
veh_trn_df_ext.shape

Remove duplicates

In [ ]:
duplicate_list = veh_trn_df_ext.columns[veh_trn_df_ext.columns.isin(per_trn_df_ext.columns)].drop(['ID','VEH_NO'])
duplicate_list

In [ ]:
veh_trn_df_ext.drop(duplicate_list, axis=1,inplace=True)
veh_trn_df_ext.shape

In [ ]:
per_veh_joined_trn_df = pd.merge(per_trn_df_ext, veh_trn_df_ext, on=['ID','VEH_NO'], how='left')

In [ ]:
per_veh_joined_trn_df.shape

In [ ]:
n_per_veh = per_veh_joined_trn_df.shape[0]

n_batch = 10000
for i in np.arange(np.ceil(n_per_veh/n_batch).astype(np.int64), dtype=np.int64):
    if i == 0:
        temp = pd.read_csv('./train/joint_person_vehicle_train_ext.csv', nrows=n_batch, index_col=0)
        temp.fillna(0, inplace=True)
        per_veh_joined_trn_df = temp
    else:
        temp = pd.read_csv('./train/joint_person_vehicle_train_ext.csv', \
                               skiprows= range(1, i*n_batch+1),\
                               nrows=n_batch, index_col=0)
        temp.fillna(0, inplace=True)
        per_veh_joined_trn_df = pd.concat([per_veh_joined_trn_df, temp])

Write to file

In [ ]:
if ifWrite:
    per_veh_joined_trn_df.to_csv('./train/joint_person_vehicle_train_ext.csv')

In [ ]:
%xdel veh_trn_df_ext
%xdel per_trn_df_ext
%xdel per_veh_joined_trn_df

Write to sqlite

In [ ]:
ifWrite = True

In [ ]:
if ifWrite:
    conn = sqlite3.connect("./train/joint_accident_person_vehicle.db")
    n_batch = 1000

    for i in np.arange(np.ceil(n_per_veh/n_batch).astype(np.int64), dtype=np.int64):
        if i == 0:
            temp = pd.read_csv('./train/joint_person_vehicle_train_ext.csv', nrows=n_batch, index_col=0)
            temp.fillna(0, inplace=True)
            temp.to_sql("person_vehcile", conn, if_exists= "replace", index=True)
        else:
            if i!= np.ceil(n_per_veh/n_batch).astype(np.int64)-1:
                temp = pd.read_csv('./train/joint_person_vehicle_train_ext.csv', \
                               skiprows= range(1, i*n_batch+1),\
                               nrows=n_batch, index_col=0)
                temp.fillna(0, inplace=True)
                temp.to_sql("person_vehcile", conn, if_exists= "append", index=True)
            else:
                temp = pd.read_csv('./train/joint_person_vehicle_train_ext.csv', \
                               skiprows= range(1, i*n_batch+1),\
                               index_col=0)
                temp.fillna(0, inplace=True)
                temp.to_sql("person_vehcile", conn, if_exists= "append", index=True)
    conn.close()

### Test data

In [ ]:
per_tst_df_ext = pd.read_csv('./test/person_test_ext.csv', index_col=0)
per_tst_df_ext.shape

In [ ]:
veh_tst_df_ext = pd.read_csv('./test/vehicle_test_ext.csv', index_col=0)
veh_tst_df_ext.shape

In [ ]:
per_veh_joined_tst_df = pd.merge(per_tst_df_ext, veh_tst_df_ext, on=['ID','VEH_NO'], how='left')
per_veh_joined_tst_df.shape

In [ ]:
per_veh_joined_tst_df.fillna(0, inplace=True)

In [ ]:
if ifWrite:
    per_veh_joined_tst_df.to_csv('./test/joint_person_vehicle_test_ext.csv')

Total number of accident ID included for given (PER_NO, VEH_NO)

mat_perno_vehno = joined_trn_df['ID'].groupby([joined_trn_df['PER_NO'], joined_trn_df['VEH_NO']]).count().unstack()
type(mat_perno_vehno)
sns.set(style="white", font_scale=1.5)
f= plt.figure(figsize=(11, 9))
ax = plt.gca()
### Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

### Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(mat_perno_vehno.fillna(0,axis=0), cmap=cmap, vmax=np.log10(np.max(np.max(mat_perno_vehno))+1),
            square=True, xticklabels=5, yticklabels=5,
            linewidths=.5, cbar_kws={"shrink": .5}, ax=ax)

cax = plt.gcf().axes[-1]
cax.tick_params(labelsize=15)

%xdel joined_trn_df
joined_trn_df = pd.merge(acc_trn_df_ext, per_trn_df_ext, on='ID', how='inner')